In [1]:
import pandas as pd
import numpy as np
import json
from statsmodels.stats.contingency_tables import mcnemar
from statsmodels.stats.contingency_tables import cochrans_q
from scipy.stats import combine_pvalues
from scipy.stats import norm

In [2]:
def read_json_answers(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)

    answers = []
    if isinstance(data, list):
        for obj in data:
            if isinstance(obj, dict) and "answer" in obj:
                answers.append(obj["answer"])
    return answers

def read_unstructured_parsed(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)

    answers = []
    if isinstance(data, list):
        for obj in data:
            if isinstance(obj, dict) and "parsed" in obj:
                answers.append(obj["parsed"])
    return answers

In [3]:
# Structured answers
json_standard_answers = read_json_answers("llc_results/test_function_0.json")
json_artist_answers = read_json_answers("llc_results/test_function_artist_0.json")
json_chef_answers = read_json_answers("llc_results/test_function_chef_0.json")
json_detective_answers = read_json_answers("llc_results/test_function_detective_0.json")
json_judge_answers = read_json_answers("llc_results/test_function_judge_0.json")
# Unstructured answers
unstructured_standard_answers = read_unstructured_parsed("llc_results/test_unstructured_parsed_0.json")
unstructured_artist_answers = read_unstructured_parsed("llc_results/test_unstructured_artist_parsed_0.json")
unstructured_chef_answers = read_unstructured_parsed("llc_results/test_unstructured_chef_parsed_0.json")
unstructured_detective_answers = read_unstructured_parsed("llc_results/test_unstructured_detective_parsed_0.json")
unstructured_judge_answers = read_unstructured_parsed("llc_results/test_unstructured_judge_parsed_0.json")

In [4]:
with open("data/llc_test_revised.json", "r") as f:
    label = json.load(f)
unstructured_standard_results = []
unstructured_artist_results = []
unstructured_chef_results = []
unstructured_detective_results = []
unstructured_judge_results = []

json_standard_results = []
json_artist_results = []
json_chef_results = []
json_detective_results = []
json_judge_results = []

for i in range(len(unstructured_standard_answers)):
    label_tmp = label[i]['answer']
    # Unstructured answers
    if label_tmp.lower() == unstructured_standard_answers[i].lower():
        unstructured_standard_results.append(1)
    else:
        unstructured_standard_results.append(0)
        
    if label_tmp.lower() == unstructured_artist_answers[i].lower():
        unstructured_artist_results.append(1)
    else:
        unstructured_artist_results.append(0)
        
    if label_tmp.lower() == unstructured_chef_answers[i].lower():
        unstructured_chef_results.append(1)
    else:
        unstructured_chef_results.append(0)
        
    if label_tmp.lower() == unstructured_detective_answers[i].lower():
        unstructured_detective_results.append(1)
    else:
        unstructured_detective_results.append(0)
        
    if label_tmp.lower() == unstructured_judge_answers[i].lower():
        unstructured_judge_results.append(1)
    else:
        unstructured_judge_results.append(0)
    
    # Structured answers
    if label_tmp.lower() == json_standard_answers[i].lower():
        json_standard_results.append(1)
    else:
        json_standard_results.append(0)   
        
    if label_tmp.lower() == json_artist_answers[i].lower():
        json_artist_results.append(1)
    else:
        json_artist_results.append(0) 
        
    if label_tmp.lower() == json_chef_answers[i].lower():
        json_chef_results.append(1)
    else:
        json_chef_results.append(0) 
        
    if label_tmp.lower() == json_detective_answers[i].lower():
        json_detective_results.append(1)
    else:
        json_detective_results.append(0) 
        
    if label_tmp.lower() == json_judge_answers[i].lower():
        json_judge_results.append(1)
    else:
        json_judge_results.append(0) 

In [5]:
# Test whether output format significantly influences LLMs' generation
print(f"Averaged accuracy of unstructured format with diverse instruction: {format((sum(unstructured_standard_results)+sum(unstructured_artist_results)+sum(unstructured_chef_results)+sum(unstructured_detective_results)+sum(unstructured_judge_results))/(5*len(unstructured_standard_results)),'.3f')}")
print(f"Averaged accuracy of JSON format with diverse instruction: {format((sum(json_standard_results)+sum(json_artist_results)+sum(json_chef_results)+sum(json_detective_results)+sum(json_judge_results))/(5*len(json_standard_results)),'.3f')}")
data_subsets = [
    # Stratum of control on standard instruction
    (np.array(json_standard_results), np.array(unstructured_standard_results)),
    # Stratum of control on artist instruction
    (np.array(json_artist_results), np.array(unstructured_artist_results)),
    # Stratum of control on chef instruction
    (np.array(json_chef_results), np.array(unstructured_chef_results)),
    # Stratum of control on detective instruction
    (np.array(json_detective_results), np.array(unstructured_detective_results)),
    # Stratum of control on judge instruction
    (np.array(json_judge_results), np.array(unstructured_judge_results)),
]

p_values = []

for i, (correct_D, correct_E) in enumerate(data_subsets, 1):
    # Build contingency table for this stratum
    table = np.zeros((2, 2), dtype=int)
    for d, e in zip(correct_D, correct_E):
        table[d, e] += 1

    # Run McNemar's test
    result = mcnemar(table, exact=True)
    p_values.append(result.pvalue)

    # print(f"Stratum {i} contingency table:\n{table}")
    # print(f"Stratum {i} McNemar p-value: {result.pvalue:.5f}\n")

# Combine p-values using Stouffer’s method with correlation adjustment
# Convert p-values to Z-scores (two-sided)
z_scores = norm.isf(np.array(p_values) / 2)  # inverse survival function

# Assume weights (e.g., uniform or by number of samples per stratum)
weights = np.ones_like(z_scores)
# Assume correlation between strata (due to shared samples)
rho = 0.3
k = len(p_values)
R = np.full((k, k), rho)
np.fill_diagonal(R, 1)  # correlation matrix

# Stouffer's Z with correlation adjustment
numerator = np.sum(weights * z_scores)
denominator = np.sqrt(np.dot(weights, R @ weights))
z_combined = numerator / denominator
p_combined = 2 * norm.sf(abs(z_combined))

# print(f"Combined Stouffer Z: {z_combined:.3f}")
print(f"Combined p-value across strata (Stouffer’s method): {p_combined:.3f}")

Averaged accuracy of unstructured format with diverse instruction: 0.940
Averaged accuracy of JSON format with diverse instruction: 0.971
Combined p-value across strata (Stouffer’s method): 0.046


In [6]:
# Test whether instruction significantly influences LLMs' generation
print(f"Averaged accuracy of unstructured & JSON format with standard instruction: {format((sum(unstructured_standard_results)+sum(json_standard_results))/(2*len(unstructured_standard_results)),'.3f')}")
print(f"Averaged accuracy of unstructured & JSON format with intervened instruction: {format((sum(unstructured_artist_results)+sum(unstructured_chef_results)+sum(unstructured_detective_results)+sum(unstructured_judge_results)+sum(json_artist_results)+sum(json_chef_results)+sum(json_detective_results)+sum(json_judge_results))/(8*len(json_standard_results)),'.3f')}")

stratum_json = np.array([json_standard_results, json_artist_results, json_chef_results, json_detective_results, json_judge_results]).transpose()
stratum_unstructured = np.array([unstructured_standard_results, unstructured_artist_results, unstructured_chef_results, unstructured_detective_results, unstructured_judge_results]).transpose()
data_strata = [stratum_json, stratum_unstructured]

# Run Cochran's Q test on each stratum
p_values = []
# print("Cochran's Q test per stratum:")
for i, data in enumerate(data_strata):
    result = cochrans_q(data)
    p_values.append(result.pvalue)
    # print(f"  Stratum {i+1}: Q = {result.statistic:.4f}, p = {result.pvalue:.5f}")

# Combine p-values using Stouffer’s method with correlation adjustment
# Convert p-values to Z-scores (two-sided)
z_scores = norm.isf(np.array(p_values) / 2)  # inverse survival function

# Assume weights (e.g., uniform or by number of samples per stratum)
weights = np.ones_like(z_scores)
# Assume correlation between strata (due to shared samples)
rho = 0.3
k = len(p_values)
R = np.full((k, k), rho)
np.fill_diagonal(R, 1)  # correlation matrix

# Stouffer's Z with correlation adjustment
numerator = np.sum(weights * z_scores)
denominator = np.sqrt(np.dot(weights, R @ weights))
z_combined = numerator / denominator
p_combined = 2 * norm.sf(abs(z_combined))

# print(f"Combined Stouffer Z: {z_combined:.3f}")
print(f"Combined p-value across strata (Stouffer’s method): {p_combined:.3f}")

Averaged accuracy of unstructured & JSON format with standard instruction: 0.957
Averaged accuracy of unstructured & JSON format with intervened instruction: 0.955
Combined p-value across strata (Stouffer’s method): 0.249
